In [1]:
%matplotlib inline
import numpy as np
import os, string
from matplotlib import pyplot as plt
import tensorflow as tf
import seaborn as sns

states = ('Rainy', 'Sunny')
 
observations = ('walk', 'shop', 'clean')
 
start_probability = {'Rainy': 0.6, 'Sunny': 0.4}
 
transition_probability = {
    'Rainy' : {'Rainy': 0.7, 'Sunny': 0.3},
    'Sunny' : {'Rainy': 0.4, 'Sunny': 0.6},
    }
 
emission_probability = {
    'Rainy' : {'walk': 0.1, 'shop': 0.4, 'clean': 0.5},
    'Sunny' : {'walk': 0.6, 'shop': 0.3, 'clean': 0.1},
}

stateNum = 2
estateNum = 3
p0=np.mat([0.6,0.4])
T = np.mat([[0.7,0.3],[0.4,0.6]])
E = np.mat([[0.1,0.4,0.5],[0.6,0.3,0.1]])
obchain = np.array([0,1,2])
print(p0)
print(T)
print(E)

[[ 0.6  0.4]]
[[ 0.7  0.3]
 [ 0.4  0.6]]
[[ 0.1  0.4  0.5]
 [ 0.6  0.3  0.1]]


In [2]:
with tf.device('/cpu:0'):
    with tf.variable_scope("zh", reuse=None):
        start = tf.placeholder('float',p0.shape)
        #print(start.get_shape())
        tT = tf.placeholder('float',T.shape) 
        tE = tf.placeholder('float',E.shape)
        #y = tf.placeholder('float',[obchain.shape[0],estateNum])
        y0 = tf.placeholder('float',[1,estateNum])
        y1 = tf.placeholder('float',[1,estateNum])
        y2 = tf.placeholder('float',[1,estateNum])
        state = []
        estate = []
        
        pick0 = tf.get_variable(initializer=tf.ones([1,stateNum])/stateNum,name='pick0')
        state0 = tf.multiply(start,pick0)#tf.get_variable(initializer=tf.ones([1,stateNum])/stateNum,name='state0')
        state0 = tf.minimum(tf.maximum(state0,0.0),1.)
        state0 /= tf.reduce_sum(state0)
        estate0 = tf.matmul(state0,tE)
        pick1 = tf.get_variable(initializer=tf.ones([1,stateNum])/stateNum,name='pick1')
        state1 = tf.multiply(tf.matmul(state0,tT),pick1)
        state1 = tf.minimum(tf.maximum(state1,0.0),1.)
        state1 /= tf.reduce_sum(state1)
        estate1 = tf.matmul(state1,tE)
        pick2 = tf.get_variable(initializer=tf.ones([1,stateNum])/stateNum,name='pick2')
        state2 = tf.multiply(tf.matmul(state1,tT),pick2)
        state2 = tf.minimum(tf.maximum(state2,0.0),1.)
        state2 /= tf.reduce_sum(state2)
        estate2 = tf.matmul(state2,tE)
        loss = tf.reduce_sum(tf.multiply(estate0,y0))
        loss *= tf.reduce_sum(tf.multiply(estate1,y1))
        loss *= tf.reduce_sum(tf.multiply(estate2,y2))
        loss *= -1.
#         for i in range(0,obchain.shape[0]):
#             latentChainProb.append(tf.get_variable(initializer=tf.random_uniform([1,2],0.,1.0),name='latentChainProb%d'%i))

#         for i in range(0,obchain.shape[0]):
#             latentChainProb[i] = tf.minimum(tf.maximum(latentChainProb[i],0.0),1.)
#             latentChainProb[i] = latentChainProb[i]/tf.reduce_sum(latentChainProb[i])
#             state = tf.multiply(state,latentChainProb[i])
#             #print(state.get_shape(), tT.get_shape())
#             state = tf.minimum(tf.maximum(tf.matmul(state,tT),0.0),1.)
#             estate = tf.minimum(tf.maximum(tf.matmul(state,tE),0.),1.)
#             #estate = tf.cast(tf.argmax(estate,dimension=1),tf.float32)
#             #print(estate.get_shape())
#             estate = estate/tf.reduce_sum(estate)
#             print(estate.dtype,estate.get_shape())
#             chainProb.append(estate)

        #epredict = tf.concat(estate,0)
        #print(epredict.get_shape())
        #loss = tf.nn.l2_loss(y1-estate[0])+tf.nn.l2_loss(estate[1]-y2)+tf.nn.l2_loss(y3-estate[2])#tf.nn.l2_loss(epredict-y)
        global_step = tf.Variable(0, name = 'global_step',trainable=False)
        starter_learning_rate = 0.1
        learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,500, 0.95, staircase=True)
        train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss=loss,global_step = global_step) 
    
init = tf.global_variables_initializer ()
# Launch the graph.
config = tf.ConfigProto(allow_soft_placement = True)
sess = tf.Session(config = config)
sess.run(tf.global_variables_initializer())  

In [3]:
import sys,time
y_=np.zeros([obchain.shape[0],estateNum],np.int)
for i in range(0,obchain.shape[0]):
    y_[obchain[i],i] = 1
print(y_)
for i in range(1000):
    #train
    _,epre0,epre1,epre2,curloss,lr = sess.run([train_op,estate0,estate1,estate2,loss,learning_rate], feed_dict={y0: y_[0,:].reshape(1,3),y1:y_[1,:].reshape(1,3),y2:y_[2,:].reshape(1,3), tT: T, tE:E,start:p0})
    if i%100 == 0:
        print(i,epre0,epre1,epre2,curloss,lr)
        #sys.stdout.write('\r epoch : %d' % i)
        #sys.stdout.flush()
        #time.sleep(0.2)
        

[[1 0 0]
 [0 1 0]
 [0 0 1]]
0 [[ 0.30000001  0.36000001  0.34      ]] [[ 0.31000003  0.35800004  0.33200002]] [[ 0.31300002  0.3574      0.32960001]] -0.035399 0.1
100 [[ 0.60000002  0.30000001  0.1       ]] [[ 0.1         0.40000001  0.5       ]] [[ 0.1         0.40000001  0.5       ]] -0.12 0.1
200 [[ 0.60000002  0.30000001  0.1       ]] [[ 0.1         0.40000001  0.5       ]] [[ 0.1         0.40000001  0.5       ]] -0.12 0.1
300 [[ 0.60000002  0.30000001  0.1       ]] [[ 0.1         0.40000001  0.5       ]] [[ 0.1         0.40000001  0.5       ]] -0.12 0.1
400 [[ 0.60000002  0.30000001  0.1       ]] [[ 0.1         0.40000001  0.5       ]] [[ 0.1         0.40000001  0.5       ]] -0.12 0.1
500 [[ 0.60000002  0.30000001  0.1       ]] [[ 0.1         0.40000001  0.5       ]] [[ 0.1         0.40000001  0.5       ]] -0.12 0.095
600 [[ 0.60000002  0.30000001  0.1       ]] [[ 0.1         0.40000001  0.5       ]] [[ 0.1         0.40000001  0.5       ]] -0.12 0.095
700 [[ 0.60000002  0.3000000

In [5]:
print(sess.run(state0,feed_dict={y0: y_[0,:].reshape(1,3),y1:y_[1,:].reshape(1,3),y2:y_[2,:].reshape(1,3), tT: T, tE:E,start:p0}))
print(sess.run(state1,feed_dict={y0: y_[0,:].reshape(1,3),y1:y_[1,:].reshape(1,3),y2:y_[2,:].reshape(1,3), tT: T, tE:E,start:p0}))
print(sess.run(state2,feed_dict={y0: y_[0,:].reshape(1,3),y1:y_[1,:].reshape(1,3),y2:y_[2,:].reshape(1,3), tT: T, tE:E,start:p0}))
#print(sess.run(latentChainProb[2]))

[[ 0.  1.]]
[[ 1.  0.]]
[[ 1.  0.]]


In [8]:
help(tf.concat)

Help on function concat in module tensorflow.python.ops.array_ops:

concat(values, axis, name='concat')
    Concatenates tensors along one dimension.
    
    Concatenates the list of tensors `values` along dimension `axis`.  If
    `values[i].shape = [D0, D1, ... Daxis(i), ...Dn]`, the concatenated
    result has shape
    
        [D0, D1, ... Raxis, ...Dn]
    
    where
    
        Raxis = sum(Daxis(i))
    
    That is, the data from the input tensors is joined along the `axis`
    dimension.
    
    The number of dimensions of the input tensors must match, and all dimensions
    except `axis` must be equal.
    
    For example:
    
    ```python
    t1 = [[1, 2, 3], [4, 5, 6]]
    t2 = [[7, 8, 9], [10, 11, 12]]
    tf.concat([t1, t2], 0) ==> [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
    tf.concat([t1, t2], 1) ==> [[1, 2, 3, 7, 8, 9], [4, 5, 6, 10, 11, 12]]
    
    # tensor t3 with shape [2, 3]
    # tensor t4 with shape [2, 3]
    tf.shape(tf.concat([t3, t4], 0)) ==> [

In [5]:
a=np.matmul(np.multiply(p0,T),E)
print(a)
print(a.sum(axis=0))

[[ 0.114  0.204  0.222]
 [ 0.168  0.168  0.144]]
[ 0.282  0.372  0.366]


In [6]:
print(0.282 + 0.372 + 0.366)

1.02
